In [1]:
import importlib
import full_model
import numpy as np

In [2]:
full_model.eng = full_model.start_matlab()


Starting MATLAB engine...
-------------------------
Elapsed: 11.63 sec


In [3]:
images_norm = full_model.loadmat("./saved/yaleB.mat")['X']
labels = full_model.loadmat("./saved/yaleB.mat")['Y'].reshape(-1)

In [4]:
from skopt.space import Real, Integer

space  = [Real(10**-2, 10**0, "log-uniform", name='lr_pretrain'),
          Real(10**-3, 10**-1, "log-uniform", name='lr'),
          Real(10**0, 10**2, "log-uniform", name='alpha1'),
          Integer(2, 3, name='maxIter1'),
          Real(10**0, 10**2, "log-uniform", name='alpha2'),
          Integer(2, 3, name='maxIter2'),]

In [5]:
from skopt.utils import use_named_args

@use_named_args(space)
def objective(**params):
    try:
        #return 1-full_model.run_model(images_norm, labels, seed, ep_prtr, ep, **params)[0]
        return params['lr_pretrain']*params['lr']*params['alpha1']*params['alpha2']
    except Exception as ex:
        if(type(ex) == KeyError):
            raise ex
        return 1

In [6]:
import gp
from skopt import dummy_minimize
from skopt.plots import plot_convergence
import time

def optimize(function, iterations, epochs_pretrain, epochs, random_seed):
    start_time = time.time()
    
    # global b/c I couldn't find a better way to directly pass these to the objective function
    global ep_prtr, ep, seed
    ep_prtr = epochs_pretrain
    ep = epochs
    seed = random_seed
    
    # kwargs b/c dummy_minimize can not take n_jobs
    kwargs = {"n_calls":iterations, "random_state":random_seed, "verbose":True, "callback":callback_func}
    if(function != dummy_minimize):
        kwargs["n_jobs"] = -1
    res = function(objective, space, **kwargs)
    end_func(res)
    opt_stats(res, start_time)
    return res

In [7]:
def opt_stats(res, start_time=None):
    plot_convergence(res, yscale="log")
    print("\n------------------")
    for i in range(10, len(res.func_vals)+1, 10):
        print("{0:d}: {1:.4f}".format(i, min(res.func_vals[0:i])))
    print("Best score: {0:.4f}".format(res.fun))
    print("Best parameters: {0:}".format(res.x))
    if(start_time is not None):
        print("Total time elapsed: {0:.2f} sec\n".format(time.time()-start_time))

In [8]:
def asdf(result):
    print(len(result.func_vals))
    print(result.random_state.get_state()[1][0])
    print(result.random_state.bytes(10))
    print(np.where(result.random_state.multinomial(1, [1/500]*500) == 1))
    print(result.random_state.bytes(10))
    if(len(result.models) > 0):
        print(result.models[-1].predict(np.zeros((1, 6))))

In [9]:
def end_func(result):
    global random_current_, random_prev_
    if("random_prev_" in globals()):
        result.random_state = random_prev_
    else:
        raise Exception("Not enough iterations")
    del random_current_
    del random_prev_

In [10]:
from copy import deepcopy

def callback_func(result):
    asdf(result)
    global random_current_, random_prev_
    if("random_current_" in globals()):
        random_prev_ = random_current_
    random_current_ = deepcopy(result.random_state)
    return False

In [11]:
def helper(result):
    global models, state, set_vars
    if(set_vars):
        # same situation with the RandomState object
        # modifying its parameters like this makes the RandomState object behave identically,
        # while keeping it at the same location in memory
        # thus, pointers in any other parts of the code will still receive this new object
        result.random_state.set_state(state)
        
        # this only needs to be performed once, after the first iteration
        set_vars = False
        asdf(result, False)
    else:
        asdf(result, True)

In [12]:
from copy import deepcopy

def reload(result, addtl_calls):
    start_time = time.time()
    args = deepcopy(result.specs['args'])
    args['n_calls'] = addtl_calls
    args['n_random_starts'] = 0
    args['x0'] = result.x_iters
    args['y0'] = result.func_vals
    args['random_state'] = result.random_state
    result_new = gp.gp_minimize(**args)
    end_func(result_new)
    opt_stats(result_new, start_time)
    return result_new

In [60]:
seeed = 1234

In [73]:
importlib.reload(gp)

<module 'gp' from 'C:\\Users\\aleks_000\\Desktop\\Mentorship\\!GitHub\\gp.py'>

In [ ]:
res1 = optimize(gp.gp_minimize, 12, 1, 1, seeed)

In [ ]:
res12 = reload(res1, 4)

In [ ]:
res2 = optimize(gp.gp_minimize, 16, 1, 1, seeed)

In [ ]:
ATTENTION - self.gains_:
[-1.02042693 -1.03237808 -0.55954234]
14

In [21]:
from skopt import load
seed_ = 1
iters_ = 50
gp_loaded = load("optims/scenario0/gp_" + str(seed_) + "_" + str(iters_) + ".opt")
dummy_loaded = load("optims/scenario0/dummy_" + str(seed_) + "_" + str(iters_) + ".opt")
forest_loaded = load("optims/scenario0/forest_" + str(seed_) + "_" + str(iters_) + ".opt")
gbrt_loaded = load("optims/scenario0/gbrt_" + str(seed_) + "_" + str(iters_) + ".opt")

In [ ]:
opt_stats(gbrt_loaded)

In [71]:
res12.func_vals

array([3.77196407e+00, 2.48504733e-03, 6.85648911e-02, 7.80830527e-01,
       7.84859376e-02, 1.54998680e-02, 1.38466320e+00, 8.88918898e-01,
       4.45292196e-02, 5.87109554e+00, 1.40929124e-03, 1.97937218e-03,
       1.60747531e-03, 2.94241614e-02, 1.00000000e-05, 3.83670628e-04])

In [72]:
res2.func_vals

array([3.77196407e+00, 2.48504733e-03, 6.85648911e-02, 7.80830527e-01,
       7.84859376e-02, 1.54998680e-02, 1.38466320e+00, 8.88918898e-01,
       4.45292196e-02, 5.87109554e+00, 1.40929124e-03, 1.97937218e-03,
       1.60747531e-03, 2.94241614e-02, 1.31557356e-03, 1.00000000e-03])

In [103]:
def test():
    state = deepcopy(res1.random_state)
    print(np.where(state.multinomial(1, [1/1000]*1000) == 1))

In [113]:
for i in range(10):
    test()
print(np.where(res1.random_state.multinomial(1, [1/1000]*1000) == 1))

(array([371], dtype=int64),)
(array([371], dtype=int64),)
(array([371], dtype=int64),)
(array([371], dtype=int64),)
(array([371], dtype=int64),)
(array([371], dtype=int64),)
(array([371], dtype=int64),)
(array([371], dtype=int64),)
(array([371], dtype=int64),)
(array([371], dtype=int64),)
(array([371], dtype=int64),)
